<a href="https://colab.research.google.com/github/ainesko/ML_HW/blob/main/RandomForest_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import numpy as np
import pandas as pd
import random
from scipy.stats import mode
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.tree import DecisionTreeClassifier

Реализация дерева решений

In [28]:
def roc(proba):
    res = []
    for i in range(len(proba)):
        try:
            res += [proba[i][1]]
        except KeyError:
            res += [0]
    return np.array(res)
def AUC(y_test, proba):
    y_pred = roc(proba)
    tpr,fpr,_ = roc_curve(y_test, y_pred)
    return auc(tpr, fpr)

In [29]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=6, min_samples_leaf=20, max_features="auto", n_estimators=10):
        self.crit = criterion
        self.max_d = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.forest = []
        for i in range(n_estimators):
          self.forest.append(DecisionTreeClassifier(criterion=criterion, max_depth=max_depth, min_samples_leaf=min_samples_leaf))
        if max_features == 'log2':
            self.func = np.log2
        else:
            self.func = np.sqrt
        self.n = n_estimators

    def fit(self, X, y, n_sample = 100):
        max_feat = int(self.func(X.shape[1]))
        self.colmn = []
        for i in range(self.n):
            Xsamp = X.sample(max_feat, axis=1)
            self.colmn.append(Xsamp.columns)
            self.forest[i].fit(Xsamp, y)

    def predict(self, X):
        prob = []
        for i in range(self.n):
            X_pr = X[self.colmn[i]]
            prob.append(self.forest[i].predict(X_pr))
        self.prob = np.array(prob).T
        return (mode(prob)[0]).T, np.array(prob).T
    
    def predict_proba(self, X):
        prob = self.predict(X)[1]
        result = []
        for i in range(len(X)):
            result.append(dict())
            for label in range(2):
                result[i][label] = 0
        for i in range(len(X)):
            temp = prob[i,:]
            for key in result[i].keys():
                result[i][key] = np.sum(temp == key) / self.n
        return result

In [30]:
rand_forest = RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_leaf=30)
X = pd.read_csv('https://raw.githubusercontent.com/ainesko/ML_HW/main/x_spam_train.csv')
y = pd.read_csv('https://raw.githubusercontent.com/ainesko/ML_HW/main/y_spam_train.csv')['Expected']
test_X = pd.read_csv('https://raw.githubusercontent.com/ainesko/ML_HW/main/x_spam_test.csv')
rand_forest.fit(X.drop('Id', axis=1), y, n_sample=200)

In [31]:
submission = pd.DataFrame(columns = ["Id", "Expected"])
submission["Id"] = test_X["Id"]
submission["Expected"] = rand_forest.predict(test_X)[0]
submission.to_csv('submission.csv', index=False)